In [23]:
from minimel import evaluate
from minimel.experiment import get_dir_params
import pandas as pd
import pathlib

fpreds = sorted(pathlib.Path("../data/wiki/simplewiki-20211120/").rglob("run*.tsv"))
fgold = pathlib.Path("../data/mewsli-en-small.tsv")

scores = evaluate(fgold, *fpreds)

dir_params = pd.DataFrame.from_records([
    {p.split("__")[0]: dict(get_dir_params(pathlib.Path(p))) for p in i.parts if "__" in p}
    for i in scores.index
])
dir_params = pd.concat(
    {k: pd.DataFrame.from_records(v) for k, v in dir_params.items()}, axis=1
)
dir_params["run"] = dir_params["run"].str.replace(".tsv", "")

train_time_logs = [f.parent.parent / "time.log" for f in fpreds]
scores["train_time"] = [float(open(f).readlines()[-1].split()[-1]) for f in time_logs]
pred_time_logs = [
    f.parent / f.name.replace("run___", "").replace(".tsv", "-time.log") for f in fpreds
]
scores["pred_time"] = [float(open(f).readlines()[-1].split()[-1]) for f in time_logs]

score_table = dir_params.join(scores.reset_index(drop=True))
score_table = score_table.drop(
    columns=[c for c, v in score_table.items() if len(set(v)) == 1]
)
score_table

Evaluating: 100%|██████████| 4/4 [00:00<00:00, 22.65it/s]


count     clean       run     micro                         macro   
  min_count min_count           precision    recall    fscore precision   
0         3         3  baseline  0.480505  0.480505  0.480505  0.428087  \
1         3         3     model  0.485092  0.485092  0.485092  0.432414   
2         4         4  baseline  0.478211  0.478211  0.478211  0.427017   
3         4         4     model  0.479931  0.479931  0.479931  0.428927   

                      pred_time  
     recall    fscore            
0  0.426892  0.426428  0.623818  
1  0.431457  0.430959  2.279957  
2  0.425819  0.425354  1.188066  
3  0.427299  0.427088  1.785688

In [3]:
print(score_table.to_latex())

\begin{tabular}{lrrrrrrrr}
\toprule
 & count & clean & \multicolumn{3}{r}{micro} & \multicolumn{3}{r}{macro} \\
 & min_count & min_count & precision & recall & fscore & precision & recall & fscore \\
\midrule
0 & 3 & 3 & 0.485092 & 0.485092 & 0.485092 & 0.432663 & 0.431705 & 0.431207 \\
1 & 4 & 4 & 0.479931 & 0.479931 & 0.479931 & 0.429174 & 0.427545 & 0.427334 \\
\bottomrule
\end{tabular}

